# Part 2

In the first step we will set up the Spark environment and load the data. WE remove empty entries and load the stopwords we will use later.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("A2-Part2-Pipeline").getOrCreate()

#load reviews and stopwords
DEBUG = True
if DEBUG:
    RAW_PATH = "hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"
else:
    RAW_PATH = "hdfs:///user/dic25_shared/amazon-reviews/full/reviewscombined.json"
stopwordsPath = "Exercise_1/stopwords.txt"

#define structure of json for faster reading
from pyspark.sql import types as T
review_schema = T.StructType([
     T.StructField("reviewerID",      T.StringType(),  True),
     T.StructField("asin",            T.StringType(),  True),
     T.StructField("reviewerName",    T.StringType(),  True),
     T.StructField("helpful",         T.ArrayType(T.IntegerType()), True),
     T.StructField("reviewText",      T.StringType(),  True),
     T.StructField("overall",         T.FloatType(),   True),
     T.StructField("summary",         T.StringType(),  True),
     T.StructField("unixReviewTime",  T.LongType(),    True),
     T.StructField("reviewTime",      T.StringType(),  True),
     T.StructField("category",        T.StringType(),  True),
 ])

#read and select category and review
df = (
    spark.read
         .schema(review_schema)
         .json(RAW_PATH)
         .selectExpr("reviewText AS text",
                     "category")
    .na.drop(subset=["text", "category"])
)

# reading the stopwords
stopwords = spark.sparkContext.textFile(stopwordsPath).collect()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4048. Attempting port 4049.
25/05/13 15:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4049. Attempting port 4050.
25/05/13 1

Here, we are creating our text processing pipeline that will transform reviews into numerical features so that we can use it as input for our Machine Learning model.
First, reviews are getting tokenized and common stopwords are getting removed.Further, we count word frequencies and convert them to TF-IDF. We also select the top 2000 most relevant terms using chi-sqaured tests.
We also convert product categories from text to numerical labels and save that  in our selected features df.

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    RegexTokenizer,
    StopWordsRemover,
    CountVectorizer,
    IDF,
    ChiSqSelector,
    StringIndexer
)

# tokenisation and lower casing with RegexTokenizer
tokenizer = RegexTokenizer(
    inputCol="text",
    outputCol="tokens",
    pattern=r"""[ \t0-9()\[\]{}.!?,;:+=\-_"'`~#@&*%€$§\\/]+""",
    gaps=True,
    toLowercase=True,
)

# stopword removal
stopper = StopWordsRemover(inputCol="tokens",stopWords = stopwords, outputCol="clean_tokens")


# vectorizing
cv = CountVectorizer(
    inputCol="clean_tokens",
    outputCol="tf",
    minDF=2,
    vocabSize=50_000, 
)

# idf weighting
idf = IDF(inputCol="tf", outputCol="tfidf")

# encode the category column from string to int
encoder = StringIndexer(inputCol="category", outputCol="label")

# select top 2000 terms by chi squared
selector = ChiSqSelector(
    numTopFeatures=2000,
    featuresCol="tfidf",
    outputCol="chi2_features",
    labelCol="label",
)

pipeline = Pipeline(stages=[tokenizer, stopper, cv, idf, encoder, selector])

Here, we are running the whole text transformation pipeline to convert reviews into features that we can use for Machine Learning. After we have fitted, we receive the vocabulary from the CountVectorizer and also the indices from the top 2000 features from ChiSqSelector. Finally, it gets mapped back to the actual words so that we receive the final list of terms.

In [3]:
# fitting pipeline
spark.conf.set("spark.sql.shuffle.partitions", "128")
df.persist()
model = pipeline.fit(df)
df.unpersist()

# extracting vocabulary and selected indices
vocab = model.stages[2].vocabulary  
selected = model.stages[-1].selectedFeatures

selected_terms = [vocab[i] for i in selected]

25/05/13 15:42:34 WARN DAGScheduler: Broadcasting large task binary with size 1243.4 KiB
25/05/13 15:42:35 WARN DAGScheduler: Broadcasting large task binary with size 1245.5 KiB


25/05/13 15:42:46 WARN DAGScheduler: Broadcasting large task binary with size 1247.5 KiB


Here we are creating the output file.

In [4]:
# saving top 2000 terms to output_ds.txt
import pathlib, os, codecs
out_file = pathlib.Path("output_ds.txt")
out_file.write_text("\n".join(selected_terms), encoding="utf-8")
print(f"Wrote {len(selected_terms)} terms to {out_file.resolve()}")

Wrote 2000 terms to /home/e12427512/Exercise_2/src/output_ds.txt


Here, we are comparing the important terms from Assignment 1 with the important words we have received above.

In [ ]:
# getting last line of line
lines = pathlib.Path("../output_dev.txt").read_text(encoding="utf-8").splitlines()
merged_vocab_line = lines[-1]  # This is the merged vocab line

# splitting merged vocabs in terms
old_terms = merged_vocab_line.strip().split()
old_set = set(old_terms)

# loading selected terms from output_ds.txt which is spark piepline
new_terms = pathlib.Path("output_ds.txt").read_text(encoding="utf-8").splitlines()
new_set = set(term.strip() for term in new_terms)

# comparing sets
common_terms = old_set & new_set
only_in_old = old_set - new_set
only_in_new = new_set - old_set

print(f"Terms in BOTH assignments: {len(common_terms)}")
print(f"Terms ONLY in Assignment 1: {len(only_in_old)}")
print(f"Terms ONLY in Spark pipeline (Assignment 2): {len(only_in_new)}")
print(f"Overlap: {len(common_terms) / len(new_set) * 100:.2f}% of Spark-selected terms are also in Assignment 1")

# Part 3

For part 3, we will reuse the fitted model from part 2 for transforming our data. We will train a model that can predict the product category from a reviews text.

In [6]:
# Reuse the fitted model from Part 2 to transform data
transformed_df = model.transform(df).select("label", "chi2_features")
display(transformed_df)

DataFrame[label: double, chi2_features: vector]

In [10]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
spark.sparkContext.setLogLevel("ERROR")   # WARN-Meldungen verschwinden lassen

Here, we will extend the pipeline by adding Normalization and a Support Vector Machine classifier. WE will use *Normalizer* with L2 norm.
Because *SVM* is a binary classifier, we will use the one vs rest strategy to handle multiple categories from our data.

In [8]:
# adding NOrmalizer with L2 norm
normalizer = Normalizer(
    inputCol="chi2_features", 
    outputCol="scaled_features", 
    p=2.0
)

# adding svm as binary classifier
binary_svm = LinearSVC(
    featuresCol="scaled_features",
    labelCol="label",
    maxIter=50,
    regParam=0.1
)

# used for multi class classification
ovr = OneVsRest(
    classifier=binary_svm,
    featuresCol="scaled_features",
    labelCol="label"
)

# pipeline from part 2 with added normalization and binary svm
full_pipeline = Pipeline(stages=[
    normalizer,
    ovr
])

# grid search for parameter optimization
param_grid = (ParamGridBuilder()
    .addGrid(binary_svm.regParam, [0.01, 0.1, 1.0])
    .addGrid(binary_svm.standardization, [True, False])
    .addGrid(binary_svm.maxIter, [10, 50])
    .build()
)

We will use a 60% training / 20% validation/ 20% testing split

In [11]:
# we will split the data in 60% training, 20% validation and 20% testing data.
train, val, test = transformed_df.randomSplit([0.6, 0.2, 0.2], seed=42)

# F1 as measure criterion
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

# using cross validation to find best parameters via 3 fold CV
cv = CrossValidator(
    estimator=full_pipeline,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    parallelism=4,
    seed=42
)

# trainin
cv_model = cv.fit(train)
best_model = cv_model.bestModel

# evaluate on test set
predictions = best_model.transform(test)
f1_score = evaluator.evaluate(predictions)
print(f"Best Model F1 Score: {f1_score:.4f}")

# printing best parameters
best_svm = best_model.stages[-1].getClassifier()
print(f"best regParam: {best_svm.getRegParam()}")
print(f"best standardization: {best_svm.getStandardization()}")
print(f"best maxIter: {best_svm.getMaxIter()}")

25/05/13 16:53:11 ERROR BlockManagerStorageEndpoint: Error in removing broadcast 47474
org.apache.spark.SparkException: Block broadcast_47474 does not exist
	at org.apache.spark.errors.SparkCoreErrors$.blockDoesNotExistError(SparkCoreErrors.scala:234)
	at org.apache.spark.storage.BlockInfoManager.blockInfo(BlockInfoManager.scala:237)
	at org.apache.spark.storage.BlockInfoManager.removeBlock(BlockInfoManager.scala:503)
	at org.apache.spark.storage.BlockManager.removeBlockInternal(BlockManager.scala:2007)
	at org.apache.spark.storage.BlockManager.removeBlock(BlockManager.scala:1973)
	at org.apache.spark.storage.BlockManager.$anonfun$removeBroadcast$3(BlockManager.scala:1959)
	at org.apache.spark.storage.BlockManager.$anonfun$removeBroadcast$3$adapted(BlockManager.scala:1959)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at org.apache.spark.storage.Blo

25/05/13 18:32:30 ERROR ContextCleaner: Error cleaning broadcast 162478
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.storage.BlockManagerMaster.removeBroadcast(BlockManagerMaster.scala:209)
	at org.apache.spark.broadcast.TorrentBroadcast$.unpersist(TorrentBroadcast.scala:351)
	at org.apache.spark.broadcast.TorrentBroadcastFactory.unbroadcast(TorrentBroadcastFactory.scala:45)
	at org.apache.spark.broadcast.BroadcastManager.unbroadcast(BroadcastManager.scala:79)
	at org.apache.spark.ContextCleaner.doCleanupBroadcast(ContextCleaner.scala:256)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$3(ContextCleaner.scala:204)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$3$adapted(ContextCleaner.scala:195)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.ContextCleaner.$anonfun$

[Stage 87072:>                                                      (0 + 2) / 2]

Best Model F1 Score: 0.5975
Best regParam: 0.01
Best standardization: True
Best maxIter: 10


In the following step, we will try a reduced feature set with a new ChiSquareSelector with 500 terms, which is a much heavier filtering than with 2000 terms. In the end, we will compare the results for both, actual filtering and heavy filtering.

In [ ]:
# new pipeline with heavy feature selection and 500 terms
heavy_selector = ChiSqSelector(
    numTopFeatures=500,
    featuresCol="tfidf",
    outputCol="chi2_features_heavy",
    labelCol="label"
)

# new normalizer for heavy features
normalizer_heavy = Normalizer(
    inputCol="chi2_features_heavy",
    outputCol="scaled_features_heavy",
    p=2.0
)

# new ovr heavy features
ovr_heavy = OneVsRest(
    classifier=binary_svm,
    featuresCol="scaled_features_heavy",
    labelCol="label"
)

# fitting the heavy selector on part 2 output
heavy_model = heavy_selector.fit(model.transform(df))
heavy_transformed = heavy_model.transform(model.transform(df)).select("label", "chi2_features_heavy")

heavy_pipeline = Pipeline(stages=[normalizer_heavy, ovr_heavy])

cv_heavy = CrossValidator(
    estimator=heavy_pipeline,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3
)

# training and evaluate
train_data = heavy_transformed.randomSplit([0.6, 0.2, 0.2], seed=42)[0]
cv_heavy_model = cv_heavy.fit(train_data)
heavy_predictions = cv_heavy_model.transform(test)
print(f"F1 Score (Heavy Filtering): {evaluator.evaluate(heavy_predictions):.4f}")